<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=5dd1d101f2bf7a3149fc89ab423c7b62fa79bfe77002f841fedbb1b921859abc
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.0
    Uninstalling protobuf-6.32.0:
      Successfully uninstalled protobuf-6.32.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-05 14:22:15
-------------------
qualified stocks: 89
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  7.27 L
Current:  1.40 C
-------------------
Today PnL: -10.91 K (-0.08%)
Current PnL: -18.17 L (-12.22%)
CY Booked + Current PnL: -8.31 L (-5.59%)
-------------------
Total profit:  2.89 L
Total loss:  -21.07 L
-------------------
Total Booked + Current PnL: 18.67 L (15.34%)
Total Booked PnL: 36.84 L (30.27%)
Curr Year Booked PnL: 9.86 L (7.05%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.19 C
Est FTT PnL: 78.89 L (56.4%)
Deployed:  1.22 C
Current:  1.40 C
CAGR/XIRR %: 8.78%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,DIXON,18940.15,-28.50,71.0,H-MC,7.44,143056.0,28120.0,8469.0,0.17,24.47,5.92,31.83,81.0,3.32,1.08,48.28,X40N,ATH,IT
4,APOLLOHOSP,8285.00,-13.75,60.0,H-LC,10.06,172293.0,16116.0,9976.0,-0.87,10.32,5.79,16.71,22.0,1.62,1.30,29.56,X40N,BTT,HEALTHCARE
79,TTKPRESTIG,770.00,105.34,65.0,M-SC,6.36,88477.0,-12300.0,12396.0,-0.01,-12.21,14.01,0.09,245.0,-0.99,0.67,16.24,OX40N,NTT,DURABLES
64,SBIN,863.00,-13.25,46.0,M-LC,4.49,213895.0,12416.0,14802.0,-0.28,6.16,6.92,13.51,60.0,0.84,1.61,19.54,XY25,NTT,BANKS
51,NATIONALUM,247.44,-44.49,73.0,H-MC,0.59,111507.0,11003.0,19636.0,1.80,10.95,17.61,30.49,79.0,0.56,0.84,46.90,MH,ATH,METALS


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
37,ICICIPRULI,790.00,-21.34,37.0,H-MC,2.20,135250.0,-538.0,43294.0,-0.22,-0.40,32.01,31.48,107.0,-0.01,1.02,11.67,X40,ATH,INSURANCE
31,HAVELLS,2069.16,-0.23,61.0,H-MC,1.92,247338.0,-393.0,75463.0,0.05,-0.16,30.51,30.30,92.0,-0.01,1.87,13.11,X40,ATH,ELECTRICAL
36,ICICIGI,2260.25,-20.58,41.0,H-MC,4.14,136619.0,486.0,30644.0,-1.35,0.36,22.43,22.86,91.0,0.02,1.03,13.01,X40,ATH,INSURANCE
53,PGHH,17973.08,-30.02,58.0,H-MC,4.98,202500.0,1680.0,67088.0,-0.18,0.84,33.13,34.25,80.0,0.03,1.53,6.08,X40,ATH,FMCG
88,WIPRO,420.00,-13.79,41.0,M-LC,6.95,151429.0,484.0,108968.0,-0.30,0.32,71.96,72.51,53.0,0.00,1.14,6.36,XR,NTT,IT


In [11]:
# top 5 to enter based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,HCLTECH,1943.91,-1.94,33.0,H-LC,9.96,223788.0,-18136.0,81414.0,-1.24,-7.50,36.38,26.15,8.0,-0.22,1.69,5.75,X40,ATH,IT
41,INFY,2275.00,-21.03,39.0,H-LC,10.97,307806.0,-5082.0,176773.0,-1.22,-1.62,57.43,54.87,3.0,-0.03,2.32,4.92,X40,BTT,IT
45,JIOFIN,387.00,-1.80,40.0,H-LC,13.99,217175.0,-231.0,53729.0,0.26,-0.11,24.74,24.61,48.0,-0.00,1.64,54.65,XY24,BTT,FINANCE
76,TCS,4998.00,-28.04,41.0,H-LC,14.39,286672.0,-59284.0,183126.0,-1.49,-17.14,63.88,35.80,1.0,-0.32,2.16,1.56,X40,BTT,IT
1,ABB,7934.00,-41.23,44.0,H-LC,13.12,245808.0,-15811.0,135022.0,-0.81,-6.04,54.93,45.57,7.0,-0.12,1.85,4.59,AR,NTT,ELECTRICAL


In [12]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,LTIM,7230.20,-5.78,52.0,H-LC,1.92,233572.0,-12579.0,91794.0,-0.37,-5.11,39.30,32.18,16.0,-0.14,1.76,29.09,X200,ATH,IT
52,NESTLEIND,1377.00,-8.10,63.0,H-LC,2.55,283000.0,17574.0,39224.0,-0.26,6.62,13.86,21.40,11.0,0.45,2.13,13.70,XY25,NTT,FMCG
43,ITC,452.00,-38.34,48.0,H-LC,3.05,199170.0,-968.0,21849.0,-2.07,-0.48,10.97,10.44,4.0,-0.04,1.50,5.05,X40,NTT,FMCG
19,CIPLA,1795.00,-19.78,51.0,H-LC,5.55,213597.0,9097.0,32317.0,-1.19,4.45,15.13,20.25,10.0,0.28,1.61,13.57,X40N,BTT,PHARMA
85,VBL,671.64,-13.27,44.0,H-LC,7.17,312301.0,-3541.0,116207.0,-3.12,-1.12,37.21,35.67,5.0,-0.03,2.36,12.58,X40N,ATH,FMCG


In [13]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
66,SHALBY,327.00,1021.21,72.0,M-SC,3.75,152474.0,-28669.0,74133.0,12.13,-15.83,48.62,25.10,235.0,-0.39,1.15,22.19,XY24,NTT,HEALTHCARE
84,VALIANTORG,838.00,-317.09,51.0,H-SC,9.63,92274.0,-41012.0,125603.0,5.00,-30.77,136.12,63.47,139.0,-0.33,0.70,54.67,XR,NTT,CHEMICALS
7,ATULAUTO,844.00,3932.43,78.0,M-SC,9.69,125635.0,-21733.0,73547.0,4.57,-14.75,58.54,35.16,236.0,-0.30,0.95,28.32,XY24,NTT,AUTO
17,CAMS,5211.76,-5.38,50.0,H-SC,5.46,107523.0,5519.0,38407.0,2.00,5.41,35.72,43.06,122.0,0.14,0.81,23.45,X40N,ATH,MISC
86,VOLTAS,1918.49,2.28,64.0,H-MC,3.99,214590.0,22848.0,73175.0,2.00,11.92,34.10,50.08,99.0,0.31,1.62,19.52,XY25,ATH,AC


In [14]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
68,SIS,528.00,2154.92,40.0,H-SC,5.67,60673.0,-20417.0,29087.0,-3.50,-25.18,47.94,10.69,156.0,-0.70,0.46,22.96,OX40N,NTT,MISC
85,VBL,671.64,-13.27,44.0,H-LC,7.17,312301.0,-3541.0,116207.0,-3.12,-1.12,37.21,35.67,5.0,-0.03,2.36,12.58,X40N,ATH,FMCG
21,COFFEEDAY,80.00,-41.65,58.0,L-SC,36.71,86189.0,-27360.0,67408.0,-2.88,-24.10,78.21,35.27,268.0,-0.41,0.65,110.95,XR,NTT,HOTELS
43,ITC,452.00,-38.34,48.0,H-LC,3.05,199170.0,-968.0,21849.0,-2.07,-0.48,10.97,10.44,4.0,-0.04,1.50,5.05,X40,NTT,FMCG
54,QUESS,986.00,-41.60,37.0,M-SC,41.99,58024.0,-6982.0,157912.0,-1.72,-10.74,272.15,232.18,198.0,-0.04,0.44,0.00,XY24,NTT,MISC


In [15]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,TTKPRESTIG,770.0,105.34,65.0,M-SC,6.36,88477.0,-12300.0,12396.0,-0.01,-12.21,14.01,0.09,245.0,-0.99,0.67,16.24,OX40N,NTT,DURABLES
39,INDIGOPNTS,1408.0,116.52,52.0,M-SC,3.52,142005.0,-32554.0,32590.0,-0.66,-18.65,22.95,0.02,221.0,-1.00,1.07,23.52,OX40N,NTT,PAINTS
46,KANSAINER,340.0,-68.34,53.0,H-SC,3.50,218070.0,-51597.0,87926.0,-0.39,-19.13,40.32,13.47,138.0,-0.59,1.64,10.30,XY24,NTT,PAINTS
80,UJJIVANSFB,60.0,82.03,42.0,H-SC,13.18,115182.0,-27297.0,46821.0,-1.04,-19.16,40.65,13.70,163.0,-0.58,0.87,35.95,OX40N,NTT,BANKS
18,CERA,9475.0,-20.05,45.0,H-SC,2.78,114030.0,-29773.0,56525.0,-0.47,-20.70,49.57,18.60,149.0,-0.53,0.86,25.84,OX40N,NTT,CERAMICS


In [16]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TCS,4998.00,-28.04,41.0,H-LC,14.39,286672.0,-59284.0,183126.0,-1.49,-17.14,63.88,35.80,1.0,-0.32,2.16,1.56,X40,BTT,IT
41,INFY,2275.00,-21.03,39.0,H-LC,10.97,307806.0,-5082.0,176773.0,-1.22,-1.62,57.43,54.87,3.0,-0.03,2.32,4.92,X40,BTT,IT
43,ITC,452.00,-38.34,48.0,H-LC,3.05,199170.0,-968.0,21849.0,-2.07,-0.48,10.97,10.44,4.0,-0.04,1.50,5.05,X40,NTT,FMCG
85,VBL,671.64,-13.27,44.0,H-LC,7.17,312301.0,-3541.0,116207.0,-3.12,-1.12,37.21,35.67,5.0,-0.03,2.36,12.58,X40N,ATH,FMCG
1,ABB,7934.00,-41.23,44.0,H-LC,13.12,245808.0,-15811.0,135022.0,-0.81,-6.04,54.93,45.57,7.0,-0.12,1.85,4.59,AR,NTT,ELECTRICAL


In [17]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
33,HEROMOTOCO,5949.07,1.28,78.0,H-MC,21.10,188125.0,37212.0,20092.0,0.45,24.66,10.68,37.97,93.0,1.85,1.42,56.20,AR,ATH,AUTO
25,DIXON,18940.15,-28.50,71.0,H-MC,7.44,143056.0,28120.0,8469.0,0.17,24.47,5.92,31.83,81.0,3.32,1.08,48.28,X40N,ATH,IT
26,DMART,5201.00,-5.89,69.0,H-LC,15.50,235082.0,50601.0,19770.0,-0.66,27.43,8.41,38.14,38.0,2.56,1.77,40.94,X40N,NTT,FMCG


In [18]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
14,BLUESTARCO,2326.38,13.00,64.0,H-MC,3.38,195000.0,30330.0,37635.0,0.31,18.42,19.30,41.28,89.0,0.81,1.47,27.87,X40N,ATH,AC
86,VOLTAS,1918.49,2.28,64.0,H-MC,3.99,214590.0,22848.0,73175.0,2.00,11.92,34.10,50.08,99.0,0.31,1.62,19.52,XY25,ATH,AC
33,HEROMOTOCO,5949.07,1.28,78.0,H-MC,21.10,188125.0,37212.0,20092.0,0.45,24.66,10.68,37.97,93.0,1.85,1.42,56.20,AR,ATH,AUTO
23,DABUR,735.00,-1.85,65.0,H-MC,4.43,213798.0,16770.0,72862.0,-0.81,8.51,34.08,45.49,102.0,0.23,1.61,22.66,XY24,BTT,FMCG
17,CAMS,5211.76,-5.38,50.0,H-SC,5.46,107523.0,5519.0,38407.0,2.00,5.41,35.72,43.06,122.0,0.14,0.81,23.45,X40N,ATH,MISC


In [19]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
33,HEROMOTOCO,5949.07,1.28,78.0,H-MC,21.10,188125.0,37212.0,20092.0,0.45,24.66,10.68,37.97,93.0,1.85,1.42,56.20,AR,ATH,AUTO
51,NATIONALUM,247.44,-44.49,73.0,H-MC,0.59,111507.0,11003.0,19636.0,1.80,10.95,17.61,30.49,79.0,0.56,0.84,46.90,MH,ATH,METALS
87,WHIRLPOOL,2270.00,-39.91,60.0,M-SC,4.21,101542.0,10044.0,68703.0,0.33,10.98,67.66,86.07,223.0,0.15,0.77,47.46,XR,NTT,DURABLES
38,INDIAMART,4850.92,-51.65,45.0,H-SC,12.22,135033.0,11697.0,122070.0,-0.52,9.48,90.40,108.45,119.0,0.10,1.02,33.96,AR,ATH,MISC
88,WIPRO,420.00,-13.79,41.0,M-LC,6.95,151429.0,484.0,108968.0,-0.30,0.32,71.96,72.51,53.0,0.00,1.14,6.36,XR,NTT,IT


In [20]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.00,-41.65,58.0,L-SC,36.71,86189.0,-27360.0,67408.0,-2.88,-24.10,78.21,35.27,268.0,-0.41,0.65,110.95,XR,NTT,HOTELS
3,ALKYLAMINE,4546.37,-4.34,45.0,H-SC,15.64,89044.0,-11919.0,106452.0,-1.35,-11.81,119.55,93.63,148.0,-0.11,0.67,32.63,SR,ATH,CHEMICALS
50,MASFIN,398.61,-18.25,51.0,H-SC,12.87,93150.0,-4830.0,26436.0,-0.05,-4.93,28.38,22.05,152.0,-0.18,0.70,35.40,XR,ATH,FINANCE
51,NATIONALUM,247.44,-44.49,73.0,H-MC,0.59,111507.0,11003.0,19636.0,1.80,10.95,17.61,30.49,79.0,0.56,0.84,46.90,MH,ATH,METALS
80,UJJIVANSFB,60.00,82.03,42.0,H-SC,13.18,115182.0,-27297.0,46821.0,-1.04,-19.16,40.65,13.70,163.0,-0.58,0.87,35.95,OX40N,NTT,BANKS


In [21]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.00,-41.65,58.0,L-SC,36.71,86189.0,-27360.0,67408.0,-2.88,-24.10,78.21,35.27,268.0,-0.41,0.65,110.95,XR,NTT,HOTELS
7,ATULAUTO,844.00,3932.43,78.0,M-SC,9.69,125635.0,-21733.0,73547.0,4.57,-14.75,58.54,35.16,236.0,-0.30,0.95,28.32,XY24,NTT,AUTO
33,HEROMOTOCO,5949.07,1.28,78.0,H-MC,21.10,188125.0,37212.0,20092.0,0.45,24.66,10.68,37.97,93.0,1.85,1.42,56.20,AR,ATH,AUTO
57,RELAXO,1176.00,-31.16,63.0,H-SC,4.94,63872.0,-51666.0,86655.0,0.14,-44.72,135.67,30.28,136.0,-0.60,0.48,25.57,X40N,NTT,FOOTWEAR
26,DMART,5201.00,-5.89,69.0,H-LC,15.50,235082.0,50601.0,19770.0,-0.66,27.43,8.41,38.14,38.0,2.56,1.77,40.94,X40N,NTT,FMCG


In [22]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.82
1,25,43.77
2,50,73.96


In [23]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    37.97
LC    36.16
MC    25.91
Name: CurrAlloc%, dtype: float64

In [24]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.86
X40      15.61
X40N     14.39
XY25     10.12
AR        9.48
XR        9.07
OX40N     7.44
MH        1.77
X200      1.76
X5K       1.44
SR        1.10
Name: CurrAlloc%, dtype: float64

In [25]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    27.90
H-MC    22.90
H-SC    22.60
M-SC    13.81
M-LC     7.17
M-MC     2.68
L-SC     1.56
L-LC     1.09
L-MC     0.33
Name: CurrAlloc%, dtype: float64

In [26]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.97,0.46,33.71
IT,13.41,-16.16,73.76
FINANCE,7.41,-22.75,68.02
BANKS,7.38,-16.73,66.30
PAINTS,6.08,-14.39,39.99
MISC,5.85,-18.59,86.56
ELECTRICAL,5.70,-9.02,69.11
AUTO,4.84,-5.34,51.16
AC,3.82,4.41,27.96


In [27]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2772708.0,22
AR,1221232.0,10
XR,1092694.0,13
X40,897142.0,10
X40N,526131.0,11
OX40N,473350.0,10
XY25,408292.0,7
SR,197567.0,2
X5K,126638.0,1


In [28]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2781053.0,27
H-LC,1418780.0,16
M-SC,1415430.0,17
H-MC,1171055.0,17
M-LC,427534.0,5
M-MC,322363.0,2
L-SC,253670.0,3
L-MC,59481.0,1
L-LC,39195.0,1


In [29]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,934917.0,6
M-SC,XY24,829463.0,7
H-SC,AR,661733.0,5
H-LC,X40,609824.0,5
H-SC,XR,523482.0,6
H-MC,XY24,455110.0,4
H-LC,AR,378199.0,2
M-MC,XY24,322363.0,2
H-LC,X40N,221421.0,5


In [30]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 29.0 seconds
